### 4. 函数和生成器

__def 是一个可运行的语句，Python 在运行到此时，创建一个函数对象并将其赋值给一个变量名（函数名）。__ 也就是说，函数也是对象，所以 def 语句可以嵌套在 def 中， 甚至 if，while 语句中。

* global 声明一个模块级别的可被赋值的变量
* nonlocal 声明一个需要被赋值的外层函数变量
* lambda 创建一个对象并将其作为结果返回
* 闭包就是一个记住了外层状态信息的函数

#### 4.1 函数

__函数内的操作根据传入的参数类型变化而变化的现象，称之为多态。__

In [1]:
def add(x, y):
    return x + y

In [2]:
add(1, 2)

3

In [3]:
add('1', '2')

'12'

In [4]:
add([1, 2], [3])

[1, 2, 3]

函数内要赋值全局变量，需要 global 声明。

In [5]:
x = 'old'

def change():
    x = 'new'
    return

change()

x

'old'

In [6]:
def change2():
    global x
    x = 'new'
    return

change2()

x

'new'

函数内要赋值外层变量，需要 nonlocal 声明。

In [7]:
def outer():
    val = 'old'
    def inner():
        val = 'new'
    inner()
    return val

outer()

'old'

In [8]:
def outer2():
    val = 'old'
    def inner():
        nonlocal val
        val = 'new'
    inner()
    return val

outer2()

'new'

函数是对象，可以有属性，以此来记住状态。

In [9]:
def addTo(base):
    def nested(x):
        nested.total += x
    nested.total = base
    return nested

In [10]:
f = addTo(0)
f(1)
f.total

1

In [11]:
f(2)
f.total

3

__lambda 表达式__

In [12]:
add2 = lambda x: x + 2
add2(3)

5

In [13]:
type(add2)

function

In [14]:
func = add2
func(6)

8

In [15]:
add3nums = lambda x, y, z: x + y + z # 多个参数
add3nums(1, 2, 3)

6

In [16]:
f = lambda x='spamm', y='eggs': x + y # 默认值参数
f()

'spammeggs'

In [17]:
f = lambda x: (lambda y: x + y) # 嵌套
f(1)(2)

3

#### 4.2 生成器

##### 4.2.1 生成器函数

调用一个使用 yield 逐个返回一系列值的函数时，得到的结果为一个生成器对象，可以在其上单次遍历。

In [18]:
def squares():
    for i in range(9): yield i ** 2

s = squares()

type(s)

generator

In [19]:
list(s)

[0, 1, 4, 9, 16, 25, 36, 49, 64]

In [20]:
for i in s: print(i) # s 已被遍历，继续遍历没有返回值

In [21]:
for i in squares(): print(i, end=', ')

0, 1, 4, 9, 16, 25, 36, 49, 64, 

##### 4.2.2 生成器表达式

In [22]:
sqrs = (i**2 for i in range(9))
type(sqrs)

generator

In [23]:
for i in sqrs: print(i, end=', ')

0, 1, 4, 9, 16, 25, 36, 49, 64, 

##### 4.2.3 yield 高阶

* yield from

In [24]:
def zero_to_eight():
    yield from range(9)

for i in zero_to_eight(): print(i, end=', ')

0, 1, 2, 3, 4, 5, 6, 7, 8, 

* 使用send 进行通信

In [25]:
def gen():
    for i in range(10):
        x = yield i
        print('receive from caller: ', x)

In [26]:
G = gen()
next(G)

0

In [27]:
G.send('hello')

receive from caller:  hello


1

* yield 协程

In [28]:
def calSum():
    total = 0
    while True:
        received = yield total
        total += received

In [29]:
C = calSum()
next(C)

0

In [30]:
C.send(1)

1

In [31]:
C.send(9)

10

通过引发异常来终止

In [32]:
C.send('Error')

TypeError: unsupported operand type(s) for +=: 'int' and 'str'

In [ ]:
next(C) # 已经到达 StopIteration

### 5. 作用域

__LEGB 原则__，变量依次查找范围:

1. Local, 局部作用域
2. External, 外层 def 或 lambda 的局部作用域
3. Global, 全局作用域
4. BuiltIn, 内置作用域

**应该最少化全局变量，最少化跨文件修改变量。**

### 6. 类

1. class 是可以运行的语句，class 语句内的赋值语句会创建数据属性，内嵌的 def 会创建方法属性，在其顶层的赋值语句，会被附加到这个类上，为所有实例共享。

2. 类本身可以创建实例对象，而类本身也是一个对象，它是 type 类型的一个实例，所有类继承自 object。

In [33]:
class A: pass

a = A()

type(a)

__main__.A

In [34]:
type(A)

type

#### 6.1 类的继承

1. Inheritor
2. Replacer
3. Extender
4. Provider

#### 6.2 运算符重载

##### 6.2.1 索引和分片

In [35]:
class MyList:
    
    # 构造函数
    def __init__(self, thelist=None):
        self.list = [] if not thelist else thelist
        
    # 索引和分片
    def __getitem__(self, index):
        return self.list[index]
    
    def __setitem__(self, index, value):
        self.list[index] = value
        
    def __repr__(self):
        return ', '.join(map(str, self.list))
        
mylist = MyList([1, 2, 3, 4, 5])

mylist

1, 2, 3, 4, 5

In [36]:
mylist[0:2] = [0, 1]
mylist

0, 1, 3, 4, 5

In [37]:
0 in mylist

True

##### 6.2.2 迭代协议

* 可迭代对象：迭代的被调对象，其 `__iter__` 方法被 iter 内置函数调用。
* 迭代器对象：可迭代对象的返回结果，在迭代过程中实际提供值的对象，它的 `__next__` 方法被 next 内置函数运行，并在结束时触发 `StopIteration` 异常

In [38]:
class MyIterList:
    '''
    一个只能单次循环的可迭代对象
    '''
    def __init__(self, thelist=None):
        self.index = 0
        self.list = [] if not thelist else thelist
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.index == len(self.list): raise StopIteration
        else:
            self.index += 1
            return self.list[self.index - 1]

mylist = MyIterList([1, 2])
myiter = iter(mylist)
next(myiter)

1

In [39]:
next(myiter)

2

In [40]:
next(myiter)

StopIteration: 

In [41]:
myiter2 = iter(mylist)
next(myiter2)

StopIteration: 

In [43]:
myiter is myiter2 # 实际上是相同的迭代器对象

True

In [44]:
class MyListIterator:
    def __init__(self, wrapped):
        self.wrapped = wrapped
        self.index = 0
        
    def __next__(self):
        if self.index == len(self.wrapped): raise StopIteration
        else:
            self.index += 1
            return self.wrapped[self.index - 1] 

class MyIterableList:
    '''
    一个返回迭代器对象的可迭代对象，支持多次迭代
    '''
    def __init__(self, thelist=None):
        self.index = 0
        self.list = [] if not thelist else thelist
    
    def __iter__(self):
        return MyListIterator(self.list)

mylist = MyIterableList([1, 2, 3])

myiter = iter(mylist)
myiter2 = iter(mylist)

myiter is myiter2 # 不同的迭代器对象

False

##### 6.2.3 成员关系，布尔测试

成员查找所调用的优先级顺序：

1. `__contains__`
2. `__iter__`
3. `__getitem__`

布尔测试率先调用 `__bool__`，其次是 `__len__`

In [45]:
class MyIters:

    def __init__(self, thelist=None):
        self.list = [] if not thelist else thelist
        
    def __bool__(self): return len(self.list) > 0
    
    def __len__(self): return len(self.list)
    
    def __contains__(self, x): return x in self.list
    
myiter = MyIters([1, 2, 3])

if myiter: print('Yes')

Yes


In [46]:
len(myiter)

3

In [47]:
0 in myiter

False

##### 6.2.4 属性访问

1. `__getattr__`, 拦截所有未定义的属性调用
2. `__setattr__`, 拦截所有属性赋值

In [48]:
class Person:
    attrs = ['name', 'age']

    def __getattr__(self, attr):
        if attr not in self.attrs: raise AttributeError(attr)
    
    def __setattr__(self, attr, value):
        if attr in self.attrs: self.__dict__[attr] = value # 如果在这里进行点号运算属性赋值就会产生递归调用
        else: raise AttributeError(attr)
            
person = Person()

person.job

AttributeError: job

In [49]:
person.age = 24
person.age

24

In [50]:
person.id = 27

AttributeError: id

##### 6.2.5 调用表达式 __call__

In [51]:
class Callee:
    
    def __call__(self, *args, **kargs):
        print('Called:', args, kargs)
        
callee = Callee()

callee(1, 2, x=3, y=4)

Called: (1, 2) {'x': 3, 'y': 4}


##### 6.2.6 slot 属性声明

通过 `__slots__` 声明限制类的属性，优化内存和速度。

1. 父类没有 slot，则子类 slot 无意义。
2. 子类没有相应 slot， 则父类 slot 无意义（不能继承）
3. 其实并不好用，除非很必要，一般不使用

In [52]:
class Sloter:
    __slots__ = ['name', 'value']

sloter = Sloter()
sloter.name = 'Jade'
sloter.name

'Jade'

In [53]:
sloter.gender = 'male'

AttributeError: 'Sloter' object has no attribute 'gender'

##### 6.2.7 __getattribute__

`__getattribute__` 比 `__getattr__` 更强大因为它拦截所有属性的访问，而不仅仅是未定义的。使用它时要避免通过属性访问传递给父类而导致递归。

In [54]:
class Person:
    
    def __init__(self, name): self._name = name

    def __getattribute__(self, attr):
        if attr == 'name': attr = '_name'
        return object.__getattribute__(self, attr) # 使用更高的父类以避免递归
            
person = Person('Jade')
person.id = 1
person.id

1

In [55]:
person.name

'Jade'